In [1]:
from glob import glob
import os
import subprocess
import nibabel as nib
import numpy as np
import pandas as pd
import utils
import scipy.ndimage
import scipy
import torch
from skimage.measure import regionprops
from monai.networks.blocks import Warp

In [2]:
def getRegionsAsCSV_antss(mask, out):
    ants_command = "ImageMath 3 " + str(out) + " LabelStats " + str(mask) + " " + str(mask)
    subprocess.call(ants_command.split(" "))  
    
def getRegionsAsCSV_scipy(mask, out):    
    img = nib.load(mask).get_fdata().astype(np.uint8)
    regions = regionprops(img)
    df = pd.DataFrame()
    for i, props in enumerate(regions):
        x, y, z = props.centroid
        x = x # + 1 # +1 because ITK-Snap is between 1-128 and Python 0-127
        y = y # + 1
        z = z # + 1
        t = 0
        label = props.label
        mass = props.area
        volume = props.area
        count = len(props.coords)
        row = {
            'x': x,
            'y': y,
            'z': z,
            't': t,
            'label': label,
            'mass': mass,
            'volume': volume,
            'count': count,
        }
        temp_df = pd.DataFrame(data=row, index=[0])
        df = pd.concat([df, temp_df], ignore_index=True)
    df.to_csv(out, index=False)
    
def get_csv_positive(csv, out):
    df = pd.read_csv(csv, sep=',', header=0)
    df_out = pd.DataFrame()
    for i in range(len(df)):
        row = {
            'x': np.abs(df['x'][i]),
            'y': np.abs(df['y'][i]),
            'z': np.abs(df['z'][i]),
            't': df['t'][i],
            'label': df['label'][i],
            'mass': df['mass'][i],
            'volume': df['volume'][i],
            'count': df['count'][i],
        }  
        temp_df = pd.DataFrame(data=row, index=[0])
        df_out = pd.concat([df_out, temp_df], ignore_index=True)
    df_out.to_csv(out, index=False)


In [3]:
out_atlas_ants = os.path.join("dataset3", "Atlas", "P56_Atlas_landmarks_FV_id_ants.csv")
out_atlas_scipy = os.path.join("dataset3", "Atlas", "P56_Atlas_landmarks_FV_id_scipy.csv")
atlas_landmark = os.path.join("dataset3", "Atlas", "P56_Atlas_landmarks_FV_id.nii.gz")
#getRegionsAsCSV_antss(atlas_landmark, out_atlas_ants)       
#getRegionsAsCSV_scipy(atlas_landmark, out_atlas_scipy) 

In [7]:
data_dir = os.path.join("/home/valentini/dev/Mousenet/dataset3/Feminad_Mean/") 
landmarks = sorted(glob(os.path.join(data_dir, "Landmarks", "*.nii.gz")))
landmarks_id = sorted(glob(os.path.join(data_dir, "Landmarks", "*_id.nii.gz")))
aff_mats = sorted(glob(os.path.join(data_dir, 'MRI', "*.mat")))
def_warps = sorted(glob(os.path.join(data_dir, 'MRI', "*_warp.nii.gz")))
def_invwarps = sorted(glob(os.path.join(data_dir, 'MRI', "*_warp_inverse.nii.gz")))
for i in range(len(landmarks)):
    print(i, end='\r')
    out_id_ants = os.path.join(data_dir, "Landmarks", landmarks[i].split('/')[-1].split('.')[0] + '_og.csv')
    out_idpos_ants = os.path.join(data_dir, "Landmarks", landmarks[i].split('/')[-1].split('.')[0] + '_ogpos.csv')
    getRegionsAsCSV_antss(landmarks[i], out_id_ants)  
    get_csv_positive(out_id_ants, out_idpos_ants)

    name = landmarks[i].split('/')[-1].split('.')[0]
    name = name.split('_')[0] + '_' + name.split('_')[1] + '_'

    aff_mat = aff_mats[i]
    def_warp = def_warps[i] 
    def_invwarp = def_invwarps[i]
    
    out_affine_csv = os.path.join(data_dir, "Landmarks", landmarks_id[i].split('/')[-1].split('.')[0] + '_aff.csv')
    ants_command = "antsApplyTransformsToPoints -d 3 -i " + str(out_id_ants) + " -o " + str(out_affine_csv) + " -t [" + str(aff_mat) + ",1]"
    subprocess.call(ants_command.split(" "))  
    out_affinepos_csv = os.path.join(data_dir, "Landmarks", landmarks_id[i].split('/')[-1].split('.')[0] + '_affpos.csv')
    get_csv_positive(out_affine_csv, out_affinepos_csv)

    out_atlas_def = os.path.join(data_dir, "Landmarks", landmarks_id[i].split('/')[-1].split('.')[0] + '_atlasfvdef.csv')
    ants_command = "antsApplyTransformsToPoints -d 3 -i " + str(out_atlas_ants) + " -o " + str(out_atlas_def) + " -t " + str(def_warp)
    subprocess.call(ants_command.split(" "))

    out_atlas_defpos = os.path.join(data_dir, "Landmarks", landmarks_id[i].split('/')[-1].split('.')[0] + '_atlasfvdefpos.csv')
    get_csv_positive(out_atlas_def, out_atlas_defpos)


In [ ]:
x = 0.0000000000
if not x:
    print('xd')